In [3]:
# BERT Zero Shot performs the best on English comments with 83% accuracy. and I have already this document
# For Dutch comments I am applying the fine tuned BERT model on all the Dutch comments collected.
# Imports
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm

#  Loading my fine-tuned model
model_path = "./bert-finetuned1"  
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

#  Load Dutch comments
df = pd.read_csv("dutch_comments_filtered_combined.csv")
df = df.dropna(subset=["Cleaned Comment Text"])
texts = df["Cleaned Comment Text"].astype(str).tolist()

#  Inference Function for the correct label sentiment
def predict_sentiment(texts, batch_size=32):
    results = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i:i + batch_size]
        tokens = tokenizer(batch_texts, padding=True, truncation=True, max_length=128, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**tokens)
            predictions = torch.argmax(outputs.logits, dim=1).cpu().numpy()
            # Map: 0 → -1, 1 → 0, 2 → 1 (reverse of training label mapping)
            label_map_reverse = {0: -1, 1: 0, 2: 1}
            mapped = [label_map_reverse[i] for i in predictions]
            results.extend(mapped)
    return results

#  Predict
df["predicted_sentiment"] = predict_sentiment(df["Cleaned Comment Text"].tolist())

# Save output
df.to_csv("dutch_comments_with_sentiment_score_final.csv", index=False)
print(" Saved as 'dutch_comments_with_sentiment_score_final.csv'")

 29%|████████████                              | 30/105 [00:11<00:28,  2.60it/s]


KeyboardInterrupt: 

In [5]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("dutch_comments_with_sentiment_score_final.csv")

# Showing the Head of Dutch sentiment
print(df.head())

       Car Model Language     Video ID         Author          Published At  \
0  Tesla Model Y    Dutch  E5mfhe-Q6lE  @BartHuitsing  2023-06-26T11:38:09Z   
1  Tesla Model Y    Dutch  E5mfhe-Q6lE  @werner134897  2023-03-11T06:08:18Z   
2  Tesla Model Y    Dutch  E5mfhe-Q6lE    @ronnie9187  2023-01-17T15:07:13Z   
3  Tesla Model Y    Dutch  E5mfhe-Q6lE      @jote2275  2022-11-06T15:39:27Z   
4  Tesla Model Y    Dutch  E5mfhe-Q6lE      @Maszzmic  2022-01-16T20:44:26Z   

             Updated At  Like Count  \
0  2023-06-26T11:38:09Z         1.0   
1  2023-03-11T06:10:00Z         0.0   
2  2023-01-17T15:07:13Z         0.0   
3  2022-11-06T15:39:27Z         0.0   
4  2022-01-16T20:44:26Z         0.0   

                                        Comment Text  \
0  Eén van de belangrijkste voordelen van de Y in...   
1  Nadruk op hogere prijs en gewicht is mijns inz...   
2  Het valt me op dat er lakschade aan de voorkan...   
3  Electrische auto&#39;s zijn enorm groot en duu...   
4  <a href

In [7]:
import pandas as pd

# Load the English comments file
df = pd.read_csv("english_comments_with_sentiment_score.csv")

# Define the mapping function
def map_sentiment(score):
    if score in [1.0, 2.0]:
        return -1
    elif score == 3.0:
        return 0
    elif score in [4.0, 5.0]:
        return 1
    else:
        return None

# Apply the mapping
df['predicted_sentiment'] = df['sentiment_score'].apply(map_sentiment)

# Drop the original score column
df = df.drop(columns=['sentiment_score'])

df.to_csv("english_comments_with_sentiment_score_final1.csv", index=False)

# Preview
print(df.head(20))

        Car Model Language     Video ID               Author  \
0   Tesla Model Y  English  PZ8NPeYFPCY              @carwow   
1   Tesla Model Y  English  PZ8NPeYFPCY    @jamesskinner1902   
2   Tesla Model Y  English  PZ8NPeYFPCY      @erichalfbee503   
3   Tesla Model Y  English  PZ8NPeYFPCY          @prayasdash   
4   Tesla Model Y  English  PZ8NPeYFPCY          @ooweesaler   
5   Tesla Model Y  English  PZ8NPeYFPCY            @lakshh._   
6   Tesla Model Y  English  PZ8NPeYFPCY  @andreasnilsson1579   
7   Tesla Model Y  English  PZ8NPeYFPCY         @sergiyg2537   
8   Tesla Model Y  English  PZ8NPeYFPCY    @FishingSituation   
9   Tesla Model Y  English  PZ8NPeYFPCY        @hotvideo1890   
10  Tesla Model Y  English  PZ8NPeYFPCY        @raysmith1971   
11  Tesla Model Y  English  PZ8NPeYFPCY          @FuzzyAason   
12  Tesla Model Y  English  PZ8NPeYFPCY             @tibhard   
13  Tesla Model Y  English  PZ8NPeYFPCY    @lisanmunim-gd4jp   
14  Tesla Model Y  English  PZ8NPeYFPCY 

In [9]:
import pandas as pd

# Load the two processed files
df_eng = pd.read_csv('/Users/feysal/Downloads/english_comments_with_sentiment_score_final1.csv', engine='python', on_bad_lines='skip')
df_dutch = pd.read_csv("dutch_comments_with_sentiment_score_final.csv")

print("Columns in df_eng:")
print(df_eng.columns.tolist())

print("\nSample rows:")
print(df_eng.head())
df_eng.rename(columns={"Like Count": "like_count"}, inplace=True)
df_dutch.rename(columns={"Like Count": "like_count"}, inplace=True)


# Ensure both have necessary columns
assert 'predicted_sentiment' in df_eng.columns and 'like_count' in df_eng.columns
assert 'predicted_sentiment' in df_dutch.columns and 'like_count' in df_dutch.columns

# Concatenate them
df_combined = pd.concat([df_eng, df_dutch], ignore_index=True)

# Fill missing like counts with 0
df_combined['like_count'] = df_combined['like_count'].fillna(0)

# Convert like_count to integer
df_combined['like_count'] = df_combined['like_count'].astype(int)

# Compute weighted sentiment
df_combined['like_weighted_sentiment'] = df_combined['predicted_sentiment'] * (df_combined['like_count'] + 1)

# Preview the result
print(df_combined[['Cleaned Comment Text', 'predicted_sentiment', 'like_count', 'like_weighted_sentiment']].head())
df_combined.to_csv("combined_comments_with_like_weighted_sentiment.csv", index=False)


Columns in df_eng:
['Car Model', 'Language', 'Video ID', 'Author', 'Published At', 'Updated At', 'Like Count', 'Comment Text', 'Cleaned Comment Text', 'predicted_sentiment']

Sample rows:
       Car Model Language     Video ID             Author  \
0  Tesla Model Y  English  PZ8NPeYFPCY            @carwow   
1  Tesla Model Y  English  PZ8NPeYFPCY  @jamesskinner1902   
2  Tesla Model Y  English  PZ8NPeYFPCY    @erichalfbee503   
3  Tesla Model Y  English  PZ8NPeYFPCY        @prayasdash   
4  Tesla Model Y  English  PZ8NPeYFPCY        @ooweesaler   

           Published At            Updated At  Like Count  \
0  2021-12-07T11:35:32Z  2021-12-07T11:35:32Z       508.0   
1  2024-12-20T10:37:45Z  2024-12-20T10:37:45Z         0.0   
2  2024-12-17T10:22:32Z  2024-12-17T10:22:32Z         0.0   
3  2024-11-30T11:09:04Z  2024-11-30T11:09:04Z         0.0   
4  2024-11-27T09:30:12Z  2024-11-27T09:30:12Z         0.0   

                                        Comment Text  \
0  What should the nex

In [1]:
import pandas as pd

# Load the RDW data with semicolon delimiter and disable low memory warnings
df_rdw = pd.read_csv('/Users/feysal/Downloads/Open_Data_RDW__Gekentekende_voertuigen_20250110.csv', sep=';', low_memory=False)

print(df_rdw.head())

  Kenteken,Voertuigsoort,Merk,Handelsbenaming,Vervaldatum APK,Datum tenaamstelling,Bruto BPM,Inrichting,Aantal zitplaatsen,Eerste kleur,Tweede kleur,Aantal cilinders,Cilinderinhoud,Massa ledig voertuig,Toegestane maximum massa voertuig,Massa rijklaar,Maximum massa trekken ongeremd,Maximum trekken massa geremd,Datum eerste toelating,Datum eerste tenaamstelling in Nederland,Wacht op keuren,Catalogusprijs,WAM verzekerd,Maximale constructiesnelheid,Laadvermogen,Oplegger geremd,Aanhangwagen autonoom geremd,Aanhangwagen middenas geremd,Aantal staanplaatsen,Aantal deuren,Aantal wielen,Afstand hart koppeling tot achterzijde voertuig,Afstand voorzijde voertuig tot hart koppeling,Afwijkende maximum snelheid,Lengte,Breedte,Europese voertuigcategorie,Europese voertuigcategorie toevoeging,Europese uitvoeringcategorie toevoeging,Plaats chassisnummer,Technische max. massa voertuig,Type,Type gasinstallatie,Typegoedkeuringsnummer,Variant,Uitvoering,Volgnummer wijziging EU typegoedkeuring,Vermogen massa

In [13]:
columns_to_keep = [
    "kenteken",
    "voertuigsoort",
    "merk",
    "handelsbenaming",
    "datum_eerste_toelating",
    "datum_eerste_tenaamstelling_in_nederland"
]

df_rdw_filtered = df_rdw[columns_to_keep]
df_rdw_filtered.to_csv("filtered_rdw_carmodels_only.csv", index=False)
print("✅ Filtered and saved successfully.")

KeyError: "None of [Index(['kenteken', 'voertuigsoort', 'merk', 'handelsbenaming',\n       'datum_eerste_toelating', 'datum_eerste_tenaamstelling_in_nederland'],\n      dtype='object')] are in the [columns]"

In [13]:
print(df_rdw.head(10))

  Kenteken,Voertuigsoort,Merk,Handelsbenaming,Vervaldatum APK,Datum tenaamstelling,Bruto BPM,Inrichting,Aantal zitplaatsen,Eerste kleur,Tweede kleur,Aantal cilinders,Cilinderinhoud,Massa ledig voertuig,Toegestane maximum massa voertuig,Massa rijklaar,Maximum massa trekken ongeremd,Maximum trekken massa geremd,Datum eerste toelating,Datum eerste tenaamstelling in Nederland,Wacht op keuren,Catalogusprijs,WAM verzekerd,Maximale constructiesnelheid,Laadvermogen,Oplegger geremd,Aanhangwagen autonoom geremd,Aanhangwagen middenas geremd,Aantal staanplaatsen,Aantal deuren,Aantal wielen,Afstand hart koppeling tot achterzijde voertuig,Afstand voorzijde voertuig tot hart koppeling,Afwijkende maximum snelheid,Lengte,Breedte,Europese voertuigcategorie,Europese voertuigcategorie toevoeging,Europese uitvoeringcategorie toevoeging,Plaats chassisnummer,Technische max. massa voertuig,Type,Type gasinstallatie,Typegoedkeuringsnummer,Variant,Uitvoering,Volgnummer wijziging EU typegoedkeuring,Vermogen massa

In [20]:
# Try with comma first
df_rdw = pd.read_csv("filtered_RDW_cars_only.csv", sep=',', low_memory=False)

# If it still gives you 1 column, switch to semicolon
# Now clean column names
df_rdw.columns = df_rdw.columns.str.strip().str.replace('"', '')

# Show cleaned columns
print("✅ Fixed Columns:", df_rdw.columns.tolist())

# Check for the column
print("Datum eerste tenaamstelling in Nederland" in df_rdw.columns)

# If still unsure, print matching ones
print("\n📌 Columns matching 'tenaamstelling':")
for col in df_rdw.columns:
    if 'tenaamstelling' in col.lower():
        print(col)


# Convert date on RDW
df_rdw["Datum eerste tenaamstelling in Nederland"] = pd.to_datetime(
    df_rdw["Datum eerste tenaamstelling in Nederland"],
    format="%Y%m%d",
    errors="coerce"
)
# Convert comment date
# Load your combined YouTube comments data
comments_df = pd.read_csv('/Users/feysal/Downloads/combined_comments_with_like_weighted_sentiment.csv')  # adjust the filename if needed

# Parse the comment dates
comments_df["Published At"] = pd.to_datetime(comments_df["Published At"], errors='coerce')

✅ Fixed Columns: ['Kenteken,Voertuigsoort,Merk,Handelsbenaming,Vervaldatum APK,Datum tenaamstelling,Bruto BPM,Inrichting,Aantal zitplaatsen,Eerste kleur,Tweede kleur,Aantal cilinders,Cilinderinhoud,Massa ledig voertuig,Toegestane maximum massa voertuig,Massa rijklaar,Maximum massa trekken ongeremd,Maximum trekken massa geremd,Datum eerste toelating,Datum eerste tenaamstelling in Nederland,Wacht op keuren,Catalogusprijs,WAM verzekerd,Maximale constructiesnelheid,Laadvermogen,Oplegger geremd,Aanhangwagen autonoom geremd,Aanhangwagen middenas geremd,Aantal staanplaatsen,Aantal deuren,Aantal wielen,Afstand hart koppeling tot achterzijde voertuig,Afstand voorzijde voertuig tot hart koppeling,Afwijkende maximum snelheid,Lengte,Breedte,Europese voertuigcategorie,Europese voertuigcategorie toevoeging,Europese uitvoeringcategorie toevoeging,Plaats chassisnummer,Technische max. massa voertuig,Type,Type gasinstallatie,Typegoedkeuringsnummer,Variant,Uitvoering,Volgnummer wijziging EU typegoedkeuri

KeyError: 'Datum eerste tenaamstelling in Nederland'

In [22]:
# Convert RDW registration date to month
df_rdw["month"] = df_rdw["Datum eerste tenaamstelling in Nederland"].dt.to_period("M").dt.to_timestamp()

# Count registrations per month
rdw_monthly = df_rdw.groupby("month").size().reset_index(name="registrations")

comments_df["month"] = comments_df["Published At"].dt.to_period("M").dt.to_timestamp()

# Aggregate sentiment by month
sentiment_monthly = comments_df.groupby("month")["like_weighted_sentiment"].sum().reset_index()

comments_df["month"] = comments_df["Published At"].dt.to_period("M").dt.to_timestamp()

# Aggregate sentiment by month
sentiment_monthly = comments_df.groupby("month")["like_weighted_sentiment"].sum().reset_index()


KeyError: 'Datum eerste tenaamstelling in Nederland'

In [89]:
# Merge on month
combined_df = pd.merge(rdw_monthly, sentiment_monthly, on="month", how="inner")

# Check the result
print(combined_df.head(20))


        month  registrations  like_weighted_sentiment
0  2021-11-01           3369                    198.0
1  2021-12-01          34871                   3043.0
2  2022-01-01          30039                    836.0
3  2022-02-01          22550                   4384.0
4  2022-03-01          24798                     80.0
5  2022-04-01          22133                    155.0
6  2022-05-01          23766                    981.0
7  2022-06-01          30740                   -517.0
8  2022-07-01          22947                     64.0
9  2022-08-01          24856                     14.0
10 2022-09-01          27330                      8.0
11 2022-10-01          30947                   2561.0
12 2022-11-01          30201                     15.0
13 2022-12-01          33991                   4933.0
14 2023-01-01          36300                   1422.0
15 2023-02-01          31749                     33.0
16 2023-03-01          41685                     47.0
17 2023-04-01          33374

In [3]:
import pandas as pd


# Read the CSV file into a DataFrame
df_rdw = pd.read_csv('/Users/feysal/Downloads/Open_Data_RDW__Gekentekende_voertuigen_20250521.csv')


# Convert to datetime
df_rdw["Datum eerste toelating"] = pd.to_datetime(df_rdw["Datum eerste toelating"], format="%Y%m%d", errors="coerce")
df_rdw["Datum eerste tenaamstelling in Nederland"] = pd.to_datetime(df_rdw["Datum eerste tenaamstelling in Nederland"], format="%Y%m%d", errors="coerce")

# Extract only year and month (YYYY-MM format)
df_rdw["Toelating YYYY-MM"] = df_rdw["Datum eerste toelating"].dt.strftime("%Y-%m")
df_rdw["Tenaamstelling YYYY-MM"] = df_rdw["Datum eerste tenaamstelling in Nederland"].dt.strftime("%Y-%m")

#  preview

print(df_rdw[df_rdw["Merk"].str.upper() == "MERCEDES-BENZ"].head(20))

    Kenteken Voertuigsoort           Merk     Handelsbenaming  \
131   GVJ50D  Personenauto  MERCEDES-BENZ             C 300 e   
153   GKS84F  Personenauto  MERCEDES-BENZ             C 300 e   
187   GTK50R  Personenauto  MERCEDES-BENZ             E 300 e   
188   GTT61N  Personenauto  MERCEDES-BENZ             CLA 180   
223   GTG41T  Personenauto  MERCEDES-BENZ            EQS 450+   
240   GRL32B  Personenauto  MERCEDES-BENZ               C 180   
267   GTG27T  Personenauto  MERCEDES-BENZ            EQB 250+   
277   GVJ78D  Personenauto  MERCEDES-BENZ             E 300 E   
298   GGV12X  Personenauto  MERCEDES-BENZ             B 250 E   
322   GVK02J  Personenauto  MERCEDES-BENZ               A 180   
328   GTJ16N  Personenauto  MERCEDES-BENZ    GLC 400 E 4MATIC   
344   GSK15G  Personenauto  MERCEDES-BENZ           CLA 250 E   
374   GTD54L  Personenauto  MERCEDES-BENZ    GLC 300 E 4MATIC   
384   GTT02N  Personenauto  MERCEDES-BENZ             A 250 e   
392   GVK14J  Personenaut

In [7]:
# Mapping from known Handelsbenaming model keywords to classes only for Mercedes as mercedes only says A B C for its car models
def assign_car_model(row):
    if row["Merk"].upper() == "BMW" and "IX1" in row["Handelsbenaming"].upper():
        return "BMW X1"
    if row["Merk"].upper() == "AUDI" and "Q4" in row["Handelsbenaming"].upper():
        return "AUDI Q4 E-TRON"
    if row["Merk"].upper() == "MERCEDES-BENZ":
        model = row["Handelsbenaming"].upper()
        mercedes_class_mapping = {
            "A ": "A-CLASS",
            "C ": "C-CLASS",
            "E ": "E-CLASS"
        }
        for prefix, cls in mercedes_class_mapping.items():
            if model.startswith(prefix):
                return f"MERCEDES-BENZ {cls}"
        return "MERCEDES-BENZ UNKNOWN"
    else:
        return f"{row['Merk']} {row['Handelsbenaming']}"

df_rdw["Car Model"] = df_rdw.apply(assign_car_model, axis=1)

In [9]:
print(df_rdw[df_rdw["Car Model"].notna()].head(20))

   Kenteken Voertuigsoort        Merk     Handelsbenaming  \
0    GTJ33L  Personenauto       SKODA              KODIAQ   
1    GRV95H  Personenauto     HYUNDAI                KONA   
2    GSZ75J  Personenauto     LYNK&CO        Lynk & Co 01   
3    GSZ74P  Personenauto      TOYOTA  TOYOTA YARIS CROSS   
4    GTD08H  Personenauto      TOYOTA       TOYOTA AYGO X   
5    GVG38R  Personenauto  VOLKSWAGEN                POLO   
6    GTV13K  Personenauto        FORD            EXPLORER   
7    GVG15H  Personenauto      TOYOTA      TOYOTA COROLLA   
8    GVG50N  Personenauto         BMW        X3 xDrive30e   
9    GTV12X  Personenauto  VOLKSWAGEN               T-ROC   
10   GTV10K  Personenauto       MAZDA              MAZDA3   
11   GTV09T  Personenauto        OPEL               CORSA   
12   GSH74L  Personenauto        FORD                Kuga   
13   GTZ07L  Personenauto         BMW                218I   
14   GTV09S  Personenauto         BMW         I4 EDRIVE35   
15   GVH24P  Personenaut

In [11]:
comments_df = pd.read_csv("combined_comments_with_like_weighted_sentiment.csv")
standard_models = comments_df["Car Model"].dropna().unique()
standard_models = [m.upper() for m in standard_models]

def normalize_rdw_car_model(rdw_model):
    if pd.isna(rdw_model):
        return None
    rdw_model = rdw_model.upper()
    for standard_model in standard_models:
        if standard_model in rdw_model:
            return standard_model
    return rdw_model

df_rdw["Car Model"] = df_rdw["Car Model"].apply(normalize_rdw_car_model)

In [15]:
# Load the combined comments data again
comments_df = pd.read_csv("combined_comments_with_like_weighted_sentiment.csv")

# Get unique car models from both
rdw_models = set(df_rdw["Car Model"].dropna().unique())
comment_models = set(comments_df["Car Model"].dropna().unique())

# See which RDW models match the comment data
matching_models = rdw_models.intersection(comment_models)
non_matching_models = rdw_models - comment_models

print("✅ Matching models:", matching_models)
print("❌ Non-matching models:", non_matching_models)
# seeing the results you can see due to uppercase/lower case it only could match BMW X3 etc so I am going to fix that in later code

✅ Matching models: {'.', ' L', 'BMW X1', 'BMW X3', 'BMW X5'}
❌ Non-matching models: {'NIESMANN-BISCHOFF FLAIR 920EK', 'CHAUSSON V690 PREMIUM', 'AUDI S6', 'FORSTER FA 699 HB', 'VOLKSWAGEN CADDY MAXI AMF-BRUNS', 'ROLLER TEAM KRONOS 284/286 TL/P', 'CHAUSSON V697', 'BENIMAR BENIVAN 119', 'MAZDA CX-3', 'CHAUSSON 720', 'GLOBECAR GLOBECAR SUMMIT 640', 'BMW 520 XDRIVE', 'JAGUAR F-PACE', 'CHALLENGER 357 FORD PREMIUM M20', 'KARMANN MOBIL DAVIS 600', 'AUDI E-TRON GT', 'BMW 220D ACTIVE TOURER', 'AUDI A1 CITY CARVER', 'CAPRON T69LC/T459', 'AUDI Q5 SPORTBACK 50 TFSI E', 'DONKERVOORT GTO-RS', 'BENTLEY FLYING SPUR AZURE V8', 'POESSL ROADCAR 540', 'HYMER YOSEMITE', 'CHALLENGER 358EB', 'VOLKSWAGEN TRANSPORTER', 'GENESIS GV80', 'KNAUS R67', 'BMW 230E XDRIVE ACTIVE TOURER', 'BMW M140I', 'FIAT/NIESMANN+BISCHOFF ARTO 88EK', 'POSSL PÖSSL SUMMIT 640', 'FORSTER FV 541 HB VANTASY', 'BUERSTNER BA7001', 'VOLKSWAGEN KNAUS TOURER VAN 500MQ', 'ADRIA CORAL 670 SC', 'DETHLEFFS FO02 = JUSTGOT7055EB', 'KIA XCEED GPL', '

In [17]:
print(df_rdw[["Merk", "Handelsbenaming", "Car Model","Tenaamstelling YYYY-MM"]].dropna().head(60))

          Merk         Handelsbenaming                       Car Model  \
0        SKODA                  KODIAQ                    SKODA KODIAQ   
1      HYUNDAI                    KONA                    HYUNDAI KONA   
2      LYNK&CO            Lynk & Co 01                    LYNK & CO 01   
3       TOYOTA      TOYOTA YARIS CROSS              TOYOTA YARIS CROSS   
4       TOYOTA           TOYOTA AYGO X                     TOYOTA AYGO   
5   VOLKSWAGEN                    POLO                 VOLKSWAGEN POLO   
6         FORD                EXPLORER                   FORD EXPLORER   
7       TOYOTA          TOYOTA COROLLA                  TOYOTA COROLLA   
8          BMW            X3 xDrive30e                          BMW X3   
9   VOLKSWAGEN                   T-ROC                VOLKSWAGEN T-ROC   
10       MAZDA                  MAZDA3                    MAZDA MAZDA3   
11        OPEL                   CORSA                      OPEL CORSA   
12        FORD                    Kuga

In [15]:
valid_models = comments_df["Car Model"].dropna().unique()
rdw_valid_count = df_rdw["Car Model"].isin(valid_models).sum()

print(f"✅ {rdw_valid_count} RDW rows have Car Models that match comment data.")
# Due to lowercase and uppercase, I code that in the next steps

✅ 78360 RDW rows have Car Models that match comment data.


In [23]:
# Select only the needed columns
filtered_rdw = df_rdw[["Merk", "Handelsbenaming", "Car Model", "Tenaamstelling YYYY-MM"]].dropna()

# Save to CSV
filtered_rdw.to_csv("filtered_rdw_for_forecasting.csv", index=False)

# Drop 'Merk' and 'Handelsbenaming' columns, then sort by 'Car Model'
final_rdw = filtered_rdw.drop(columns=["Merk", "Handelsbenaming"]).sort_values(by="Car Model")

# Save the final version
final_rdw.to_csv("final_rdw_car_model_tenaamstelling.csv", index=False)

In [25]:
import pandas as pd

# Load the final RDW data
rdw_path = "final_rdw_car_model_tenaamstelling.csv"
df_rdwfinal = pd.read_csv(rdw_path)

# Define the list of 40 car models
selected_models = [
    "AUDI Q4 E-TRON", "BMW X1", "BMW X3", "BMW X5", "FIAT 500", "FORD KUGA",
    "HONDA CR-V", "HYUNDAI I10", "HYUNDAI KONA", "KIA NIRO", "KIA PICANTO",
    "KIA SPORTAGE", "LYNK & CO 01", "MERCEDES-BENZ A-CLASS", "MERCEDES-BENZ C-CLASS",
    "MERCEDES-BENZ E-CLASS", "MINI COOPER", "OPEL CORSA", "PEUGEOT 2008",
    "PEUGEOT 208", "RENAULT CLIO", "RENAULT MÉGANE", "RENAULT SCENIC", "SKODA ENYAQ",
    "SKODA FABIA", "SKODA OCTAVIA", "SKODA SUPERB", "TESLA MODEL 3", "TESLA MODEL Y",
    "TOYOTA AYGO", "TOYOTA COROLLA", "TOYOTA YARIS", "TOYOTA YARIS CROSS",
    "VOLKSWAGEN GOLF", "VOLKSWAGEN POLO", "VOLKSWAGEN TIGUAN", "VOLKSWAGEN UP",
    "VOLVO EX30", "VOLVO XC40", "VOLVO XC60"
]

# Filter the RDW data to only include the selected models
filtered_rdw = df_rdwfinal[df_rdwfinal["Car Model"].isin(selected_models)]

# Save to the same file
filtered_path = "final_rdw_car_model_tenaamstelling.csv"
filtered_rdw.to_csv(filtered_path, index=False)

In [33]:

# Load the CSV file
df = pd.read_csv("final_rdw_car_model_tenaamstelling.csv")

# Checking how many cars we have after I filtered out every car out except the 40 car models that got sentiment analysis.
print("Number of cars:", len(df))

Number of cars: 642785


In [35]:
# Now at the final stage, merging the data together. the sentiment analysis with weighted likes and the RDW registration data
df_rdw = pd.read_csv("final_rdw_car_model_tenaamstelling.csv")
df_sentiment = pd.read_csv("prepared_comments_data_for_lightgbm_part_2.csv")
# First, standardize column names
df_sentiment = df_sentiment.rename(columns={"YYYY-MM": "month"})
df_rdw = df_rdw.rename(columns={"Tenaamstelling YYYY-MM": "month"})

# Ensure same casing
df_sentiment["Car Model"] = df_sentiment["Car Model"].str.upper()
df_rdw["Car Model"] = df_rdw["Car Model"].str.upper()

# Merge
df_merged = pd.merge(df_rdw, df_sentiment, on=["month", "Car Model"], how="left")

# Sort before lag
df_merged = df_merged.sort_values(by=["Car Model", "month"])

# Lag features
for lag in [1, 2, 3]:
    df_merged[f"lag_{lag}_registrations"] = df_merged.groupby("Car Model")["registrations"].shift(lag)
    df_merged[f"lag_{lag}_sentiment"] = df_merged.groupby("Car Model")["like_weighted_sentiment"].shift(lag)

# Create lag features
for lag in [1, 2, 3]:
    df_merged[f"lag_{lag}_registrations"] = df_merged.groupby("Car Model")["registrations"].shift(lag)
    df_merged[f"lag_{lag}_sentiment"] = df_merged.groupby("Car Model")["like_weighted_sentiment"].shift(lag)

# Extract additional date-based features
df_merged["month"] = pd.to_datetime(df_merged["month"])
df_merged["month_num"] = df_merged["month"].dt.month
df_merged["year"] = df_merged["month"].dt.year
df_merged["time_index"] = (df_merged["year"] - df_merged["year"].min()) * 12 + df_merged["month_num"]

# Drop rows with missing lag values
df_merged = df_merged.dropna(subset=["lag_1_registrations", "lag_1_sentiment"])

# Save the merged and feature-enhanced dataset
df_merged.to_csv("ready_for_lightgbm_forecasting.csv", index=False)
print(df_merged.head(10))

         Car Model      month  registrations  like_weighted_sentiment  \
1   AUDI Q4 E-TRON 2021-12-01           1202                     -5.0   
2   AUDI Q4 E-TRON 2022-01-01            119                      1.0   
3   AUDI Q4 E-TRON 2022-02-01            310                      5.0   
4   AUDI Q4 E-TRON 2022-03-01            462                     -4.0   
5   AUDI Q4 E-TRON 2022-04-01            236                     -8.0   
6   AUDI Q4 E-TRON 2022-05-01            305                     -5.0   
7   AUDI Q4 E-TRON 2022-06-01            359                     -1.0   
8   AUDI Q4 E-TRON 2022-07-01            163                     -1.0   
9   AUDI Q4 E-TRON 2022-08-01            116                     -1.0   
10  AUDI Q4 E-TRON 2022-09-01            132                      2.0   

    cumulative_like_weighted_sentiment  lag_1_registrations  lag_1_sentiment  \
1                               1543.0                 31.0             64.0   
2                               1544